# Iminuit Comparison
This notebook looks at an example IV characteristic (shot 338 - as in single_shot_analysis_magnum.ipynb) and compares the use of curve_fit to the use of iminuit in terms of fitting quality and error analysis. 

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr 
import scipy.stats as stat
import sys
import os
import glob
import re
import importlib
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.core.colours as col
import flopter.core.ivdata as iv
import flopter.core.lputils as lp
import flopter.core.constants as c
import flopter.magnum.database as ut
import flopter.core.fitters as fts
import flopter.magnum.utils as mgut

In [2]:
# Create analysed dataset metadata 

path_to_datasets = '/home/jleland/data/external/magnum/'
# path_to_analysed_datasets = 'analysed_4_downsampled'
path_to_analysed_dns_datasets = 'analysed_4_downsampled'
path_to_analysed_datasets = 'analysed_4'
os.chdir(path_to_datasets)


analysed_infos_df = mgut.get_dataset_metadata(path_to_analysed_datasets)
analysed_dns_infos_df = mgut.get_dataset_metadata(path_to_analysed_dns_datasets)

In [3]:
# adc_index = 329
adc_index = 271

In [4]:
analysed_infos_df.loc[adc_index]

shot_number                                              256
shot_timestamp                           6698686205167668224
shot_time                                2019-06-04 15:43:40
filename          analysed_4/a256_271_6698686205167668224.nc
time_len                                                5000
sweep_len                                                799
Name: 271, dtype: object

In [5]:
# shot_number = 337
# shot_number = 338
# shot_number = 339
# shot_number = 330

shot_number = 137

# shot_number = 256

shot_number = 239

In [6]:
# Get row from shot_number instead of ADC index
shot_oi = analysed_infos_df.loc[analysed_infos_df['shot_number'] == shot_number]
dns_shot_oi = analysed_dns_infos_df.loc[analysed_dns_infos_df['shot_number'] == shot_number]
print(shot_oi['filename'].values)
print(dns_shot_oi['filename'].values)

['analysed_4/a239_252_6698670651268073472.nc']
['analysed_4_downsampled/a239_252_6698670651268073472.nc']


In [7]:
analysis_ds = xr.open_dataset(shot_oi['filename'].values[0])
dns_analysis_ds = xr.open_dataset(dns_shot_oi['filename'].values[0])

metadata_ds = xr.open_dataset('all_meta_data.nc').sel(shot_number=shot_number).load()
# print(analysis_ds)

In [8]:
analysis_ds #['ts_temperature'].plot.line()

<xarray.Dataset>
Dimensions:     (direction: 2, probe: 2, sweep: 787, time: 5000)
Coordinates:
  * time        (time) float64 0.0 1e-06 2e-06 ... 0.004997 0.004998 0.004999
  * direction   (direction) object 'up' 'down'
  * probe       (probe) object 'S' 'L'
Dimensions without coordinates: sweep
Data variables:
    voltage     (probe, direction, sweep, time) float64 ...
    current     (probe, direction, sweep, time) float64 ...
    shot_time   (probe, direction, sweep, time) float64 ...
    start_time  (probe, direction, sweep) float64 ...

In [9]:
# probe = 'S'
probe = 'L'
# probe = 'B'

In [10]:
magnum_probes = lp.MagnumProbes()

In [11]:
gaussian = fts.NormalisedGaussianFitter()

In [12]:
# probe_colours = {
#     'L': '#4477AA',
#     'S': '#EE6677',
#     'B': '#228833',
#     'R': '#CCBB44'
# }
probe_colours = {
    'L': col.palettes['c'][3],
    'S': col.palettes['c'][2],
    'B': col.palettes['c'][1],
    'R': col.palettes['b'][2],
}

# Using curve_fit 
Using flopter.core.fitters to fit a 4-param fit to the sweep-averaged IV using non-linear least squares.
This section has gotten a little out of control and now isn't really anything to do with comparing iminuit and curve_fit. The first section is a recreation of analysed_dataset processing, but expanded to include the non-downsampled datasets. These are both loaded in their entirety and coarsened (instead of downsampled) with every 100 values binned and averaged - a more rigorous approach (I think) than downsampling by taking every 100th value as it means the binned values can be used in the calculation of the standard deviation when sweep averaging.

The section that follows is the prototype for the temperature minimisation algorithm, which is now implemented (as of 2020-06-16) as a method in IVData. This will be upgraded to allow the choice of minimisation parameter. 

A section follows this which tests the new methods for plotting and converting IVData objects into xarray datasets or pandas dataframes. 

The final section is a prototype for an enhanced gunn algorithm whereby the current due to sheath expansion current is subtracted to get the electron current, then a semi-log linear fit is used to extract the electron temperature. This can all then be combined to produce a synthetic/idealised IV characteristic which should (hopefully) yield an acceptable $\chi^2_\nu$

In [13]:
swp_lim_dn, swp_lim_up = mgut.find_sweep_limit(analysis_ds)
sweep_slc = mgut.find_allowed_sweeps(analysis_ds)
print(swp_lim_dn, swp_lim_up, sweep_slc)


0 657 <xarray.DataArray 'current' (sweep: 787)>
array([ True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True, 

In [16]:
# full_shot_ds = analysis_ds.sel(probe=probe).isel(sweep=slice(swp_lim_dn, swp_lim_up))
full_shot_ds = analysis_ds.sel(probe=probe).isel(sweep=sweep_slc)
full_sweep_avg_ds = full_shot_ds.mean(['sweep', 'direction'])

# Assign errors
full_sweep_avg_ds = full_sweep_avg_ds.assign({
    'd_current': full_shot_ds.std(['sweep', 'direction'])['current'], 
#     'd_voltage': full_shot_ds.std(['sweep', 'direction'])['voltage'],
    'stderr_current': full_shot_ds.std(['sweep', 'direction'])['current'] / np.sqrt(full_shot_ds['sweep'].size * full_shot_ds['direction'].size),
#     'stderr_current': full_shot_ds.std(['sweep', 'direction'])['current'] / np.sqrt(full_shot_ds.std(['sweep', 'direction'])['current'].size), 
#     'stderr_voltage': full_shot_ds.std(['sweep', 'direction'])['voltage'] / np.sqrt(full_shot_ds.std(['sweep', 'direction'])['voltage'].size),
}) 


In [17]:
print(full_shot_ds.std(['sweep', 'direction'])['current'].size)
print(full_sweep_avg_ds['current'].size)

5000
5000


In [18]:
# Comparison of the sweep range adjusted IV and the whole shot IV, with computation of errors in each bin

downsample = 100
semi_coarse_ds = analysis_ds.sel(probe=probe).coarsen(time=downsample).mean().mean('direction')
# semi_coarse_ds = semi_coarse_ds #.mean(['sweep','direction'])

fig, ax = plt.subplots(2, figsize=[8, 6])
semi_coarse_ds['current'].min('time').plot.line(x='sweep', ax=ax[0], label='Whole Shot')
semi_coarse_ds.isel(sweep=slice(swp_lim_dn, swp_lim_up))['current'].min('time').plot.line(x='sweep', ax=ax[0], label='Trimmed')
ax[0].axvline(x=swp_lim_dn, ls='--', lw=0.8, color=probe_colours[probe], label='Example sweep pos.')

# Assign 
semi_coarse_ds = semi_coarse_ds.assign({
    'd_current': analysis_ds.sel(probe=probe).coarsen(time=downsample).std()['current'], 
    'stderr_current': analysis_ds.sel(probe=probe).coarsen(time=downsample).mean()['current'] / np.sqrt(downsample),     
}) 

# plot a specific IV characteristic to get a feel for the errors (not averaged, as this would also average the errors together)
ax[1].errorbar('voltage', 'current', yerr='d_current', data=semi_coarse_ds.isel(sweep=swp_lim_dn).sel(direction='up'), 
               label=f'IV at Sweep {swp_lim_dn}', ls='none', marker='x', color=probe_colours[probe])
# ax[1].axvline(x=0, **c.AX_LINE_DEFAULTS)
ax[1].axhline(y=0, **c.AX_LINE_DEFAULTS)
# ax[1].errorbar('voltage', 'current', yerr='d_current', data=semi_coarse_ds.mean(['direction', 'sweep']))
# ax[1].errorbar('voltage', 'current', yerr='d_current', data=semi_coarse_ds.isel(sweep=slice(*mgut.find_sweep_limit(analysis_ds))).mean(['direction', 'sweep']))

ax[0].set_ylabel('Minimum Current [A]')
ax[0].set_xlabel('Sweep Number')
ax[1].set_xlabel('Probe Voltage [V]')
ax[1].set_ylabel('Probe Current [A]')

for axis in ax:
    axis.legend()
fig.tight_layout()

In [20]:
semi_coarse_ds = analysis_ds.coarsen(time=downsample).mean()
semi_coarse_ds = semi_coarse_ds.assign({
    'd_current': analysis_ds.coarsen(time=downsample).std()['current'], 
    'stderr_current': analysis_ds.coarsen(time=downsample).mean()['current'] / np.sqrt(downsample),     
}) 

In [37]:
fig, axes = plt.subplots(2, 2, figsize=[8, 6])
selected_shot = 0
# selected_shot = swp_lim_dn



for i, probe in enumerate(['L', 'S']):
    ax = axes[:,i]
    
    plot_ds = semi_coarse_ds.sel(probe=probe)

    plot_ds.mean('direction')['current'].min('time')\
        .plot.line(x='sweep', ax=ax[0], label='Whole Shot', ls=':', color='grey')
    
    plot_ds.mean('direction').sel(sweep=sweep_slc)['current'].min('time')\
        .plot.line(x='sweep', ax=ax[0], label='Trimmed', color=probe_colours[probe])
    
    ax[0].axvline(x=selected_shot, ls='--', lw=1.5, color=probe_colours[probe], 
                  label='Example sweep pos.', alpha=0.3)

    # Assign 


    # plot a specific IV characteristic to get a feel for the errors (not averaged, as this would also average the errors together)
    ax[1].errorbar('time', 'voltage', yerr='d_current', 
                   data=plot_ds.isel(sweep=selected_shot).sel(direction='down'), 
                   label=f'IV at Sweep {selected_shot}', ls='none', marker='x', color=probe_colours[probe])
#     ax[1].errorbar('voltage', 'current', yerr='d_current', 
#                    data=plot_ds.isel(sweep=selected_shot).sel(direction='up'), 
#                    label=f'IV at Sweep {selected_shot}', ls='none', marker='x', color=probe_colours[probe])
    
    ax[1].axhline(y=0, ls='-', color='black', lw=0.5)
    ax[0].set_ylabel('')
    if i == 0:
        ax[0].set_ylabel('Minimum Current [A]')
        ax[1].set_ylabel('Probe Current [A]')
    ax[0].set_xlabel('Sweep Number')
    ax[1].set_xlabel('Probe Voltage [V]')
    
    ax[0].set_title(f'Probe {probe}')
    

    for axis in ax:
        axis.legend()
fig.tight_layout()

In [22]:
# This looks at the combined average of bin, direction and sweep, and the subsequent errors.
downsample = 100

sweeps_ds = analysis_ds.sel(probe=probe).isel(sweep=slice(swp_lim_dn, swp_lim_up))
coarsening = sweeps_ds.reset_index('direction', drop=True).coarsen(time=downsample, direction=2, sweep=(swp_lim_up - swp_lim_dn))
coarse_ds = coarsening.mean().squeeze()
sweep_std_ds = coarsening.std().squeeze()

coarse_ds = coarse_ds.assign({
    'd_current': sweep_std_ds['current'], 
    'stderr_current': sweep_std_ds['current'] / np.sqrt(2 * (swp_lim_up - swp_lim_dn) * downsample),
}) 

# num_blocks = sweeps_ds['time'].size // downsample
# blocks_index = np.array([])
# for block in range(num_blocks):
#     np.append(blocks_index, np.zeros(downsample) + block)
# print(blocks_index)

print(coarse_ds)

<xarray.Dataset>
Dimensions:         (time: 50)
Coordinates:
  * time            (time) float64 4.95e-05 0.0001495 ... 0.00485 0.00495
    probe           <U1 'S'
Data variables:
    voltage         (time) float64 -49.47 -49.46 -49.43 ... -88.49 -88.45 -88.42
    current         (time) float64 0.7871 0.7859 0.7837 ... -0.4357 -0.4373
    shot_time       (time) float64 4.405 4.405 4.405 4.405 ... 4.405 4.405 4.405
    start_time      float64 4.403
    d_current       (time) float64 0.2089 0.2361 0.288 ... 0.04324 0.04125
    stderr_current  (time) float64 0.0005763 0.0006513 ... 0.0001193 0.0001138


In [25]:
# Create the downsampled dataset from the alternative analysis_ds. This creates the sweep_avg_ds from previous versions of this notebook for the downsampled data. 
dns_shot_ds = dns_analysis_ds.sel(probe=probe).isel(sweep=slice(swp_lim_dn, swp_lim_up))
dns_sweep_avg_ds = dns_shot_ds.mean(['sweep', 'direction'])

# Assign errors
dns_sweep_avg_ds = dns_sweep_avg_ds.assign({
    'd_current': dns_shot_ds.std(['sweep', 'direction'])['current'], 
    'stderr_current': dns_shot_ds.std(['sweep', 'direction'])['current'] / np.sqrt(dns_shot_ds['sweep'].size * dns_shot_ds['direction'].size)
}) 

In [26]:
fig, ax = plt.subplots(4, sharex=True)

ax[0].errorbar('voltage', 'current', yerr='stderr_current', fmt='.', data=coarse_ds, label='coarsened')
ax[1].errorbar('voltage', 'current', yerr='stderr_current', fmt='.', data=full_sweep_avg_ds, label='full')
ax[2].errorbar('voltage', 'current', yerr='stderr_current', fmt='.', data=dns_sweep_avg_ds, label='downsampled')
ax[0].legend()
ax[1].legend()
ax[2].legend()

ax[3].plot('voltage', 'stderr_current', '.', data=full_sweep_avg_ds, label='full')
ax[3].plot('voltage', 'stderr_current', '.', data=coarse_ds, label='coarsened')
ax[3].plot('voltage', 'stderr_current', '.', data=dns_sweep_avg_ds, label='downsampled')
ax[3].set_yscale('log')
ax[3].legend()


In [27]:
# sweep_avg_ds = coarse_ds
# sweep_avg_ds = full_sweep_avg_ds
sweep_avg_ds = dns_sweep_avg_ds

---

## Test for changes made to multi_fit
Temperature minimisation fitting added on 2020-05-19

In [28]:
iv_data = iv.IVData.from_dataset(sweep_avg_ds, sigma='stderr_current')
print(iv_data.multi_fit(plot_fl=False).reduced_chi2)
print(iv_data.multi_fit(plot_fl=False, minimise_temp_fl=False).reduced_chi2)

trim_iv_data = iv_data.lower_trim(16)
print(fts.FullIVFitter().fit_iv_data(trim_iv_data, sigma=trim_iv_data['sigma']).reduced_chi2)

2.4754239741320965
1.6026639780879661
569.9260283039135


In [29]:
fit_data = iv_data.multi_fit(plot_fl=True, minimise_temp_fl=False, sat_region=-65)
fit_data.reduced_chi2

1.6026639780879346

In [33]:
fig, ax = plt.subplots()
ax.plot(iv_data['V'])

---

In [24]:
fig, axes = plt.subplots(2, sharex=True)

axes[0].errorbar('voltage', 'current', yerr='d_current', data=sweep_avg_ds, linewidth=0.5)
axes[0].fill_between(
    sweep_avg_ds.voltage.values, 
    sweep_avg_ds.current.values + sweep_avg_ds.stderr_current.values, 
    sweep_avg_ds.current.values - sweep_avg_ds.stderr_current.values,
    alpha=0.5
)

axes[1].plot('voltage', 'd_current', data=sweep_avg_ds, label='st dev.')
axes[1].plot('voltage', 'stderr_current', data=sweep_avg_ds, label='st err.')
axes[1].set_yscale('log')
axes[1].legend()

In [25]:
# Attempt at plotting semilog plot

fig, axes = plt.subplots(2, sharex=True)

axes[0].errorbar('voltage', 'current', yerr='stderr_current', data=sweep_avg_ds)
axes[0].fill_between(
    sweep_avg_ds.voltage.values, 
    sweep_avg_ds.current.values + sweep_avg_ds.d_current.values, 
    sweep_avg_ds.current.values - sweep_avg_ds.d_current.values,
    alpha=0.5
)

axes[1].plot(sweep_avg_ds['voltage'], np.log(sweep_avg_ds['current'] / -0.18) + 1)
# axes[1].set_yscale('log')


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/computation.py:604: RuntimeWarning: invalid value encountered in log
  result_data = func(*input_data)


Below is a function to run a fit of an IV for a given upper index, i.e. a position past the floating potential. The idea is that this could be transformed into a fitting function which minimises the temperature while scanning over the upper_index

In [26]:

def fit_iv(sweep_avg_ds, upper_index, ax=None, multi_fit_fl=False, suppress_plotting_fl=False):
    print(upper_index)
    if ax is None and suppress_plotting_fl is False:
        fig, ax = plt.subplots()

    if sweep_avg_ds['voltage'].values[0] < sweep_avg_ds['voltage'].values[-1]:
        sweep_trim_ds = sweep_avg_ds.isel(time=slice(0,upper_index))
    else:
        sweep_trim_ds = sweep_avg_ds.isel(time=slice(sweep_avg_ds.time.size - upper_index,-1))
        
    max_voltage = sweep_trim_ds['voltage'].max('time').values
    
    if not suppress_plotting_fl:
#         fig.suptitle(f'upper_index = {upper_index}')
        fig.suptitle(r'$V_{cutoff}$ = ' + f'{max_voltage:.2g}')
        ax.errorbar(sweep_trim_ds['voltage'].values, -sweep_trim_ds['current'].values, 
        #             yerr=sweep_trim_ds['d_current'].values,
                    yerr=sweep_trim_ds['stderr_current'].values,
        #             xerr=sweep_trim_ds['d_voltage'].values,
                    linestyle='none', color='k', ecolor='k', label='Sweep-averaged IV', zorder=2)


        # Plot the whole IV in an inset axis
        inner_ax = plt.axes([0.2, 0.35, .2, .2])
        (-sweep_avg_ds.set_coords('voltage')['current']).plot(x='voltage', ax=inner_ax)
        inner_ax.axvline(x=sweep_trim_ds.max('time')['voltage'].values, **c.AX_LINE_DEFAULTS)
        inner_ax.set_title('Whole IV')
        inner_ax.set_xlabel('V')
        inner_ax.set_ylabel('I')
        inner_ax.set_xticks([])
        inner_ax.set_yticks([])

    shot_iv = iv.IVData(sweep_trim_ds['voltage'].values,
#                         -sweep_trim_ds['current'].values,
                        -(sweep_trim_ds['current'].values - 0.001166),
                        sweep_trim_ds['shot_time'].values,
#                         sigma=sweep_trim_ds['d_current'].values)
                        sigma=sweep_trim_ds['stderr_current'].values)
    # shot_iv = iv.IVData(sweep_avg_ds['voltage'].values,
    #                     -sweep_avg_ds['current'].values,
    #                     sweep_avg_ds['shot_time'].values,
    #                     sigma=sweep_avg_ds['stderr_current'].values)
    
    if multi_fit_fl:
        shot_fit = shot_iv.multi_fit(sat_region=-52, minimise_temp_fl=False, trim_to_floating_fl=False)
    else:
        fitter = fts.FullIVFitter()
        shot_fit = fitter.fit_iv_data(shot_iv, sigma=shot_iv['sigma'])
    
    fit_dens = magnum_probes.probe_s.get_density(shot_fit.get_isat(), shot_fit.get_temp(), alpha=np.radians(7.98))[0]
#     print(fit_dens)

    temp = metadata_ds.ts_temp_max.values
    dens = metadata_ds.ts_dens_max.values
    
    if not suppress_plotting_fl:
        chi_2_str = r"$\chi^2_{red}$"
        ax.plot(*shot_fit.get_fit_plottables(), label=f'Fit - T_e={shot_fit.get_temp():.3g}, n_e={fit_dens:.3g}, {chi_2_str} = {shot_fit.reduced_chi2:.3g}')

        ax.legend()
    
#     return shot_fit
    return shot_fit, shot_iv

In [29]:
shot_fits = []
shot_ivs = []
varying_indices = np.arange(35,51)
# varying_indices = np.arange(3600,5000,50)

for i in varying_indices:
    shot_fit, shot_iv = fit_iv(sweep_avg_ds, i, suppress_plotting_fl=False, multi_fit_fl=True)
    shot_ivs.append(shot_iv)
    shot_fits.append(shot_fit)

35


MultiFitError: RuntimeError occured in stage 3. 
Original error: Optimal parameters not found: The maximum number of function evaluations is exceeded.

In [28]:
gunn_f_data, gunn_iv_data = shot_iv.gunn_fit(plot_fl=True)

NameError: name 'shot_iv' is not defined

In [66]:
volts = []
temps = []
d_temps = []
chis = []
for shot_fit in shot_fits:
    volts.append(np.max(shot_fit.raw_x))
    temps.append(shot_fit.get_temp())
    d_temps.append(shot_fit.get_temp_err())
    chis.append(shot_fit.reduced_chi2)
volts = np.array(volts)
temps = np.array(temps)
d_temps = np.array(d_temps)
chis = np.array(chis)     

In [67]:
floating_interp = sweep_avg_ds.swap_dims({'time':'current'}).interp(current=0).voltage
print((floating_interp.time * 10000).round().values)

43.0


In [68]:
# Plot of fitted variables as a function of trimming index

fig, ax = plt.subplots(3, sharex=True)
ax[0].errorbar(volts, temps, yerr=d_temps)
ax[1].plot(volts, chis)
ax[1].set_yscale('log')
ax[1].set_ylabel(r'$\chi^2_{red}$')


if sweep_avg_ds['voltage'].values[0] < sweep_avg_ds['voltage'].values[-1]:
    ds_trimmed = sweep_avg_ds.isel(time=(varying_indices - 1))
else:
    ds_trimmed = sweep_avg_ds.isel(time=sweep_avg_ds.time.size - varying_indices)

ax[2].errorbar('voltage', 'current', yerr='stderr_current', data=ds_trimmed)
ax[2].fill_between(
    ds_trimmed.voltage.values, 
    ds_trimmed.current.values + ds_trimmed.stderr_current.values, 
    ds_trimmed.current.values - ds_trimmed.stderr_current.values,
    alpha=0.5
)

v_f = sweep_avg_ds.swap_dims({'time':'current'}).interp(current=0).voltage
ax[0].axvline(x=v_f, **c.AX_LINE_DEFAULTS)
ax[1].axvline(x=v_f, **c.AX_LINE_DEFAULTS)
ax[2].axvline(x=v_f, **c.AX_LINE_DEFAULTS)

### Goodness of Fit Minimisation
The below function plots a big comparison of various goodness of fit measures, and where their respective minima/maxima lie. These are:
 - $T_e$ minimum
 - $T_e \cdot \Delta T_e$ minimum
 - minimum of $|\chi^{2}_{\nu} - 1| + 1$, which is ~1 when perfect fit
 - $T_e \cdot \Delta T_e \cdot (|\chi^{2}_{\nu} - 1| + 1)$ minimum (called the "goodness of fit" parameter for lack of a better name
 - maximum of a log-normal distribution with $\mu = 1$, $\sigma = \frac{1}{T_e \cdot \Delta T_e}$ (I think)


In [69]:
fig, ax = plt.subplots(5, 2, sharex=True)

chi2_str = r'$\chi^{2}_{\nu}$'
temp_str = r'$T_e$'
d_temp_str = r'$\Delta T_e$'
chi_param_str = r'$|\chi^{2}_{\nu} - 1| + 1$'
alt_chi_param_str = r'$|\chi^{2}_{\nu} - 1|$'


temp_param = temps * d_temps
chi_param = np.abs(chis - 1) + 1
alt_chi_param = np.abs(chis - 1)
goodness_param = temp_param * chi_param
alt_goodness_param = temp_param * alt_chi_param
gaussian_param = gaussian.fit_function(np.log(chis), 1/(temps * d_temps), np.log(1))

v_f = sweep_avg_ds.swap_dims({'time':'current'}).interp(current=0).voltage

chi_index = volts[np.argmin(chis)]
chi_param_index = volts[np.argmin(chi_param)]
alt_chi_param_index = volts[np.argmin(alt_chi_param)]
goodness_index = volts[np.argmin(goodness_param)]
alt_goodness_index = volts[np.argmin(alt_goodness_param)]
gaussian_index = volts[np.argmax(gaussian_param)]
TdT_index = volts[np.argmin(temp_param)]
T_index = volts[np.argmin(temps)]

ax[0][0].errorbar(volts, temps, yerr=d_temps)
ax[0][0].set_ylabel(temp_str)
ax[0][0].axvline(x=T_index, color='blue', linestyle='--', label='T min')
ax[0][0].axvline(x=TdT_index, color='purple', linestyle='--', )
ax[0][0].axvline(x=chi_param_index, color='green', linestyle='--', )
ax[0][0].axvline(x=goodness_index, color='red', linestyle='--', )
ax[0][0].axvline(x=gaussian_index, color='orange', linestyle=':', )

ax[0][1].plot(volts, chis, label=chi2_str)
ax[0][1].axhline(y=1, **c.AX_LINE_DEFAULTS)
ax[0][1].set_yscale('log')
ax[0][1].set_ylabel(chi2_str)
ax[0][1].axvline(x=T_index, color='blue', linestyle='--',)
ax[0][1].axvline(x=TdT_index, color='purple', linestyle='--', )
ax[0][1].axvline(x=chi_param_index, color='green', linestyle='--', )
ax[0][1].axvline(x=goodness_index, color='red', linestyle='--', )
ax[0][1].axvline(x=gaussian_index, color='orange', linestyle='dotted', )

ax[1][0].plot(volts, temp_param)
ax[1][0].set_ylabel(temp_str + d_temp_str)
ax[1][0].set_yscale('log')
ax[1][0].axvline(x=TdT_index, color='purple', linestyle='--', label='T * dT')

ax[2][0].plot(volts, chi_param)
ax[2][0].set_yscale('log')
ax[2][0].set_ylabel(chi_param_str)
ax[2][0].axvline(x=chi_param_index, color='green', linestyle='--', label=chi_param_str)

ax[3][0].plot(volts, goodness_param)
ax[3][0].set_yscale('log')
ax[3][0].set_ylabel(r'$T_e . \Delta T_e . (|\chi^{2}_{\nu} - 1| + 1)$')
ax[3][0].axvline(x=goodness_index, color='red', linestyle='--', label='goodness')

ax[1][1].plot(volts, gaussian_param)
ax[1][1].set_yscale('log')
ax[1][1].set_ylabel(r'gaussian param')
ax[1][1].axvline(x=gaussian_index, color='orange', linestyle='dotted', label='gaussian ')

ax[2][1].plot(volts, alt_chi_param)
ax[2][1].set_yscale('log')
ax[2][1].set_ylabel(alt_chi_param_str)
ax[2][1].axvline(x=alt_chi_param_index, color='green', linestyle='--', label=alt_chi_param_str)

ax[3][1].plot(volts, alt_goodness_param)
ax[3][1].set_yscale('log')
ax[3][1].set_ylabel(r'$T_e . \Delta T_e . |\chi^{2}_{\nu} - 1|$')
ax[3][1].axvline(x=alt_goodness_index, color='gold', linestyle='--', label='alt_goodness')

d_temps_ratio_str = r'$\frac{\Delta T_e}{T_e}$'
ax[4][0].plot(volts, d_temps / temps)
ax[4][0].set_ylabel(d_temps_ratio_str)
ax[4][0].set_yscale('log')
# ax[1][0].axvline(x=TdT_index, color='purple', linestyle='--', label=d_temps_ratio_str)
ax[4][0].axhline(y=10, color='black', label='Threshold')


for col in ax:
    for axis in col:
        axis.axvline(x=v_f, **c.AX_LINE_DEFAULTS)
        axis.legend()

ValueError: attempt to get argmin of an empty sequence

Alternative plot using gridspec (as implemented in IVData)

In [58]:
import matplotlib.gridspec as gridspec

height_ratios = [2.5,1,1,1,1]
fig = plt.figure(constrained_layout=True)
gs = fig.add_gridspec(ncols=2, nrows=5, height_ratios=height_ratios)

ax = []

ax_big = fig.add_subplot(gs[0, :])
ax_big.errorbar('voltage', 'current', yerr='stderr_current', data=sweep_avg_ds)
ax_big.fill_between(
    sweep_avg_ds.voltage.values, 
    sweep_avg_ds.current.values + sweep_avg_ds.stderr_current.values, 
    sweep_avg_ds.current.values - sweep_avg_ds.stderr_current.values,
    alpha=0.5
)
ax_big.axvline(x=v_f, **c.AX_LINE_DEFAULTS, label=r'$V_f$')
ax_big.axvline(x=sweep_avg_ds['voltage'].isel(time=max(varying_indices)-1), linewidth=0.7, color='black', label='trim min/max')
ax_big.axvline(x=sweep_avg_ds['voltage'].isel(time=min(varying_indices)), linewidth=0.7, color='black')
ax_big.legend()

for row in range(1,5):
    ax_left = fig.add_subplot(gs[row, 0])
    ax_right = fig.add_subplot(gs[row, 1])
    ax.append([ax_left, ax_right])

ax[0][0].errorbar(volts, temps, yerr=d_temps)
ax[0][0].set_ylabel(temp_str)
ax[0][0].axvline(x=T_index, color='blue', linestyle='--', label='T min')
ax[0][0].axvline(x=TdT_index, color='purple', linestyle='--', )
ax[0][0].axvline(x=chi_param_index, color='green', linestyle='--', )
ax[0][0].axvline(x=goodness_index, color='red', linestyle='--', )

ax[0][1].plot(volts, chis, label=chi2_str)
ax[0][1].axhline(y=1, **c.AX_LINE_DEFAULTS)
ax[0][1].set_yscale('log')
ax[0][1].set_ylabel(chi2_str)
ax[0][1].axvline(x=T_index, color='blue', linestyle='--',)
ax[0][1].axvline(x=TdT_index, color='purple', linestyle='--', )
ax[0][1].axvline(x=chi_param_index, color='green', linestyle='--', )
ax[0][1].axvline(x=goodness_index, color='red', linestyle='--', )

ax[1][0].plot(volts, temp_param)
ax[1][0].set_ylabel(temp_str + d_temp_str)
ax[1][0].set_yscale('log')
ax[1][0].axvline(x=TdT_index, color='purple', linestyle='--', label='T * dT')

d_temps_ratio_str = r'$\frac{\Delta T_e}{T_e}$'
ax[1][1].plot(volts, d_temps / temps)
ax[1][1].set_ylabel(d_temps_ratio_str)
ax[1][1].set_yscale('log')
ax[1][1].axhline(y=10, color='black', label='Threshold')

ax[2][0].plot(volts, chi_param)
ax[2][0].set_yscale('log')
ax[2][0].set_ylabel(chi_param_str)
ax[2][0].axvline(x=chi_param_index, color='green', linestyle='--', label=chi_param_str)

ax[2][1].plot(volts, alt_chi_param)
ax[2][1].set_yscale('log')
ax[2][1].set_ylabel(alt_chi_param_str)
ax[2][1].axvline(x=alt_chi_param_index, color='green', linestyle='--', label=alt_chi_param_str)

ax[3][0].plot(volts, goodness_param)
ax[3][0].set_yscale('log')
ax[3][0].set_ylabel(r'$T_e . \Delta T_e . (|\chi^{2}_{\nu} - 1| + 1)$')
ax[3][0].axvline(x=goodness_index, color='red', linestyle='--', label='goodness')

ax[3][1].plot(volts, alt_goodness_param)
ax[3][1].set_yscale('log')
ax[3][1].set_ylabel(r'$T_e . \Delta T_e . |\chi^{2}_{\nu} - 1|$')
ax[3][1].axvline(x=alt_goodness_index, color='gold', linestyle='--', label='alt_goodness')

for col in ax:
    for axis in col:
        axis.axvline(x=v_f, **c.AX_LINE_DEFAULTS)
        axis.legend()

In [118]:
fig, ax = plt.subplots(2, sharex=True)

ax[0].errorbar(volts, temps, yerr=d_temps)
ax[0].set_ylabel(temp_str)
ax[0].axvline(x=T_index, color='blue', linestyle='--', label=r'$(T_e)_{min}$')
ax[0].axvline(x=v_f, **c.AX_LINE_DEFAULTS, label=r'$V_P$')
ax[0].legend()

ax[1].plot(volts, temp_param)
ax[1].set_ylabel(temp_str + r'$\cdot$' + d_temp_str)
# ax[1].set_yscale('log')
ax[1].set_xlabel(r'$V_{cutoff}$')
ax[1].axvline(x=TdT_index, color='purple', linestyle='--', label=r'$(T \cdot \Delta T)_{min}$')
ax[1].axvline(x=v_f, **c.AX_LINE_DEFAULTS, label=r'$V_P$')
ax[1].legend()


In [30]:
lowest_T_index = np.argmin(temps)
print(varying_indices[lowest_T_index], np.min(temps), chis[lowest_T_index], volts[lowest_T_index])
lowest_chi_index = np.argmin(np.abs(np.array(chis) - 1))
print(varying_indices[lowest_chi_index], chis[lowest_chi_index], temps[lowest_chi_index], volts[lowest_chi_index])

38 2.7417929887872936 0.00499694197807361 -11.377263240562195
48 0.8522767974256072 9.927175433782065 1.8940722877944929


In [205]:
# Formulation of the gaussian parameter

T = [1,2,4,6,8,10]
chi2 = np.logspace(-3, +3, 200)
# chi2_param = np.abs(chi2 - 1) + 1
chi2_param = -np.abs(chi2/(chi2 - 1))
positive_chi = chi2_param > 1
negative_chi = chi2_param < 1
# chi2_gaussian = (fts.GenericGaussianFitter().fit_function(np.log(chi2), 1, 1, np.log(1))) #/ (np.abs(chi2 - 1) + 1)
chi2_gaussian = gaussian.fit_function(np.log(chi2), 1, np.log(1))

fig, ax = plt.subplots(2)
# ax[0].plot(chi2[positive_chi], chi2_param[positive_chi], )
# ax[0].plot(chi2[negative_chi], chi2_param[negative_chi], )
# ax[0].plot(chi2, chi2_param, )
ax[0].plot(chi2, chi2_gaussian)
ax[0].set_xscale('log')

for temperature in T:
#     chi2_gaussian = (fts.GenericGaussianFitter().fit_function(np.log(chi2), 1, 1, np.log(1))) #/ (np.abs(chi2 - 1) + 1)
    chi2_gaussian = gaussian.fit_function(np.log(chi2), 2/temperature, np.log(1))

    ax[1].plot(chi2, chi2_gaussian, label=f'T={temperature}')
ax[1].set_xscale('log')
ax[1].legend()

In [88]:
T = np.linspace(0.5, 12, 200)
chi2 = np.logspace(-3, +3, 7)

fig, ax = plt.subplots()

for chi_boi in [0.5,1.0,1.5]:
    chi2_gaussian = gaussian.fit_function(np.log(chi_boi), 1/T, np.log(1))
    ax.plot(T, chi2_gaussian, label=f'chi = {chi_boi}')

ax.set_yscale('log')
ax.legend()

## Playing with the conversion between IVData objects and datasets

In [197]:
importlib.reload(c)
importlib.reload(iv)
importlib.reload(fts)

<module 'flopter.core.fitters' from '/home/jleland/coding/projects/flopter/flopter/core/fitters.py'>

In [14]:
shot_iv = iv.IVData.from_dataset(sweep_avg_ds, sigma='stderr_current')

In [15]:
isinstance(shot_iv, iv.IVData)

True

In [16]:
fig, ax = plt.subplots()
vf_index = vf_index = shot_iv.get_vf_index()

shot_iv.plot(ax=ax, trim_lines_fl=False)
shot_iv.lower_trim(vf_index).plot(ax=ax, trim_lines_fl=False)

In [17]:
upper_lim_frac = 0.2  # 10/50
lower_lim_frac = 0.1  # 5/50
step_frac = 0.02

trim_range_updist = int(upper_lim_frac * len(shot_iv['t']))
trim_range_lodist = int(lower_lim_frac * len(shot_iv['t']))
trim_range_step = max(int(step_frac * len(shot_iv['t'])), 1)


if shot_iv['V'][0] < shot_iv['V'][-1]:
    trim_range = np.arange(max(vf_index - trim_range_lodist, 0), 
                           min(vf_index + trim_range_updist, len(shot_iv['t'])),
                           trim_range_step)
else:
    trim_range = np.arange(max(vf_index - trim_range_updist, 0), 
                           min(vf_index + trim_range_lodist, len(shot_iv['t'])),
                           trim_range_step)

In [19]:
fig, ax = plt.subplots(2)

fitter = fts.FullIVFitter()

trim_fits = []
temps = []
shot_iv.plot(ax=ax[0])
for i in trim_range:
    if shot_iv['V'][0] < shot_iv['V'][-1]:
        trim_iv = shot_iv.upper_trim(i)
    else:
        trim_iv = shot_iv.lower_trim(i)
    trim_iv.plot(ax=ax[0])
    
    trim_fit = fitter.fit_iv_data(trim_iv, sigma=trim_iv['sigma'])
    
    trim_fits.append(trim_fit)
    temps.append(trim_fit.get_temp())

ax[1].plot(trim_range, temps)



In [25]:
volts = []
temps = []
d_temps = []
chis = []
for trim_fit in trim_fits:
    volts.append(np.max(trim_fit.raw_x))
    temps.append(trim_fit.get_temp())
    d_temps.append(trim_fit.get_temp_err())
    chis.append(trim_fit.reduced_chi2)

In [21]:
fig, ax = plt.subplots(2, sharex=True)
shot_iv.plot(ax=ax[0])
ax[1].plot(volts, temps)

In [22]:
trim_fits[np.argmin(temps)].reduced_chi2

1.045807622294931

In [25]:
shot_iv.multi_fit(plot_fl=True)

In [26]:
shot_iv.plot()

<ErrorbarContainer object of 3 artists>

# Enhanced Gunn Fit 


In [76]:
# iv_data = shot_iv.copy()
# iv_data = shot_ivs[-5].copy()
# iv_data = shot_ivs[-12].copy()
iv_data = shot_ivs[-1].copy()
iv_data.plot()

<ErrorbarContainer object of 3 artists>

In [77]:
# sat_region=-40
# sat_region=-20
sat_region=-65

# define a straight section and trim the iv data to it
str_sec = np.where(iv_data['V'] <= sat_region)
iv_data_ss = iv.IVData.non_contiguous_trim(iv_data, str_sec)

# needed to define the area of the straight section on a graph with a vertical line
str_sec_end = np.argmax(iv_data['V'][str_sec])

# fit & plot a straight line to the 'straight section'
sl_fitter = fts.StraightLineFitter()
fit_data_ss = sl_fitter.fit(iv_data_ss['V'], iv_data_ss['I'], sigma=iv_data_ss['sigma'])

# Extrapolate the straight line over a wider voltage range for illustrative purposes
sl_range = np.linspace(-120, 100, 100)
sl_function = fit_data_ss.fit_function(sl_range)

# Subtract the gradient of the straight section from the whole IV curve.
iv_data_corrected = iv_data.copy()
iv_data_corrected['I'] = iv_data_corrected['I'] - (fit_data_ss.get_param('m') * iv_data_corrected['V'])

simple_iv_fitter = fts.SimpleIVFitter()
fit_data_corrected = iv_data_corrected.multi_fit(sat_region=sat_region, iv_fitter=simple_iv_fitter,
                                                 plot_fl=True, minimise_temp_fl=False)

In [78]:
fig, ax = plt.subplots(3, sharex=True)

# loggable_slice = slice(25, None)
loggable_slice = slice(None, None)

# print(iv_data_corrected)
loggable_voltage = iv_data_corrected['V'][loggable_slice]
loggable_current = (-iv_data_corrected['I'] + fit_data_ss.get_param('y_0'))[loggable_slice]
# loggable_current = (-iv_data_corrected['I'] + fit_data_ss.get_param('y_0') + 0.01)[25:]

min_offset = min(loggable_current)
loggable_current -= (min_offset * 1.0)

ax[0].plot(loggable_voltage, loggable_current, label='full')
ax[1].plot(loggable_voltage, np.log(loggable_current), label='full')
ax[0].set_ylabel(r'$I_e$')
ax[1].set_ylabel(r'$\ln{I_e}$')

# fit_slc = slice(14,25)
# fit_slc = slice(6,20)
fit_slc = slice(7,16)

fittable_voltage = loggable_voltage[fit_slc]
fittable_current = loggable_current[fit_slc]

ax[0].plot(fittable_voltage, fittable_current, label='fit-section')
ax[1].plot(fittable_voltage, np.log(fittable_current), label='fit-section')

fit_indices = np.where(np.isfinite(np.log(fittable_current)))

log_f_data = sl_fitter.fit(fittable_voltage[fit_indices], np.log(fittable_current[fit_indices]))
new_temp = 1/log_f_data.get_param("m")
ax[1].plot(*log_f_data.get_fit_plottables(), label=r'$T_e=$'+f'{new_temp:.3g}')

ax[0].legend()
ax[1].legend()

mid_point_voltage = loggable_voltage[:-1] + loggable_voltage[1:] / 2
ax[2].plot(loggable_voltage, 1 / np.gradient(np.log(loggable_current)), label='full')
ax[2].plot(fittable_voltage[fit_indices], 1 / np.gradient(np.log(fittable_current[fit_indices])), label='fit-section')
ax[2].set_ylabel(r'$\frac{d \ln{I_e}}{dV}$')
ax[2].set_xlabel(r'$V$ ')
ax[2].legend()


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in log
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in log


In [66]:
print(new_temp)

2.5405123208108176


In [67]:
# The fitting from IVData's gunn_fit()

plt.figure()
plt.errorbar(iv_data['V'], iv_data['I'], yerr=iv_data['sigma'], label='Full IV', color='darkgrey',
             ecolor='lightgray')
plt.legend()
plt.xlabel(r'$V_p$ / V')
plt.ylabel(r'$I$ / A')
# plt.ylim(-0.5, 1.3)
plt.xlim(-102, 5)

plt.figure()
plt.errorbar(iv_data['V'], iv_data['I'], yerr=iv_data['sigma'], label='Full IV', color='darkgrey',
             ecolor='lightgray')
plt.plot(sl_range, sl_function, label='SE Line', color='blue', linewidth=0.5, zorder=10)
plt.legend()
plt.xlabel(r'$V_p$ / V')
plt.ylabel(r'$I$ / A')
# plt.ylim(-0.5, 1.3)
plt.xlim(-102, 5)

plt.figure()
plt.plot(sl_range, sl_function, label='SE Line', color='blue', linewidth=0.5, zorder=10)
plt.errorbar(iv_data_corrected['V'], iv_data_corrected['I'], label='Corrected IV',
             yerr=iv_data_corrected[c.SIGMA], color='darkgrey', ecolor='lightgray')
plt.legend()
plt.xlabel(r'$V_p$ / V')
plt.ylabel(r'$I$ / A')
# plt.ylim(-0.5, 1.3)
plt.xlim(-102, 5)

plt.figure()
plt.plot(sl_range, sl_function, label='SE Line', color='blue', linewidth=0.5)
plt.errorbar(iv_data_corrected['V'], iv_data_corrected['I'], label='Corrected IV',
             yerr=iv_data_corrected[c.SIGMA], color='darkgrey', ecolor='lightgray')
plt.plot(*fit_data_corrected.get_fit_plottables(), label='3 Param-Fit', zorder=10, color='r')
plt.legend()
plt.xlabel(r'$V_p$ / V')
plt.ylabel(r'$I$ / A')
# plt.ylim(-0.5, 1.3)
# plt.xlim(-102, 5)

plt.show()

In [60]:
print(log_f_data.get_param('y_0'))

1.8571852947483756


In [68]:
# Work this back into an IV characteristic

# v_f is defined above with the dataset
# v_f = iv_data.get_vf()
I_sat = fit_data_ss.get_param('y_0')
delta_I = fit_data_ss.get_param('m')
I_e = np.exp(log_f_data.get_param("y_0"))
T_e = 1 / log_f_data.get_param("m")
v = iv_data['V']

probe_current = I_sat + (delta_I * v) - (I_e * np.exp((v) / T_e))

fig, ax = plt.subplots()

iv_data.plot(ax=ax, fmt='x', color='k', zorder=-1)
red_chi = np.sum((iv_data['I'] - probe_current)**2 / (iv_data['sigma'])**2) / (len(iv_data['V']) - 4)
ax.plot(iv_data['V'], probe_current, label=r'$T_e = {:.3g}, \chi^2_{{red}} = {:.3g}$'.format(T_e, red_chi))

# fit_data = shot_fits[-5]
fit_data = shot_fits[-12]
ax.plot(*fit_data.get_fit_plottables(), label=r'$T_e = {:.3g}, \chi^2_{{red}} = {:.3g}$'.format(fit_data.get_temp(), fit_data.reduced_chi2))
ax.legend()

----

## Two temperature log-fitting

In [81]:
# Two temp linear semi-log fit
fig, ax = plt.subplots(3, sharex=True)

# loggable_slice = slice(25, None)
loggable_slice = slice(5, -1)

# print(iv_data_corrected)
loggable_voltage = iv_data_corrected['V'][loggable_slice]
loggable_current = (-iv_data_corrected['I'] + fit_data_ss.get_param('y_0'))[loggable_slice]
# loggable_current = (-iv_data_corrected['I'] + fit_data_ss.get_param('y_0') + 0.01)[25:]

min_offset = min(loggable_current)
loggable_current -= (min_offset * 1.0)

ax[0].plot(loggable_voltage, loggable_current, label='full')
ax[1].plot(loggable_voltage, np.log(loggable_current), label='full')
ax[0].set_ylabel(r'$I_e$')
ax[1].set_ylabel(r'$\ln{I_e}$')

# fit_slc = slice(14,25)
# fit_slc = slice(6,20)
fit_slc = slice(4,12)

fittable_voltage = loggable_voltage[fit_slc]
fittable_current = loggable_current[fit_slc]

ax[0].plot(fittable_voltage, fittable_current, label='fit-section')
ax[1].plot(fittable_voltage, np.log(fittable_current), label='fit-section')

fit_indices = np.where(np.isfinite(np.log(fittable_current)))

log_f_data = sl_fitter.fit(fittable_voltage[fit_indices], np.log(fittable_current[fit_indices]))
new_temp = 1/log_f_data.get_param("m")
ax[1].plot(*log_f_data.get_fit_plottables(), label=r'$T_e=$'+f'{new_temp:.3g}')

ax[0].legend()
ax[1].legend()

mid_point_voltage = loggable_voltage[:-1] + loggable_voltage[1:] / 2
ax[2].plot(loggable_voltage, 1 / np.gradient(np.log(loggable_current)), label='full')
ax[2].plot(fittable_voltage[fit_indices], 1 / np.gradient(np.log(fittable_current[fit_indices])), label='fit-section')
ax[2].set_ylabel(r'$\frac{d \ln{I_e}}{dV}$')
ax[2].set_xlabel(r'$V$ ')
ax[2].legend()


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:40: RuntimeWarning: divide by zero encountered in log


---
### Check of the calculation of $\chi^2_{\nu}$

In [192]:
# alternative calculation of chi^2
import scipy.stats as stat

numpy_chis = []
manual_chis = []
scipy_chis = []

numpy_redchis = []
manual_redchis = []
scipy_redchis = []

for shot_fit in shot_fits:
    numpy_chi = np.sum((shot_fit.raw_y - shot_fit.fit_y)**2 / (shot_fit.sigma)**2)
    numpy_chis.append(numpy_chi)
    numpy_redchis.append(numpy_chi / (len(shot_fit.raw_x) - 4))
    
    manual_chi = 0
    for i in range(len(shot_fit.raw_y)):
        manual_chi += (shot_fit.raw_y[i] - shot_fit.fit_y[i])**2 / (shot_fit.sigma[i]**2)
    manual_chis.append(manual_chi)
    manual_redchis.append(manual_chi / (len(shot_fit.raw_x) - 4))
    
    scipy_chi, p = stat.chisquare(shot_fit.raw_y, shot_fit.fit_y, ddof=3)
    scipy_chis.append(scipy_chi)
    scipy_redchis.append(scipy_chi / (len(shot_fit.raw_x) - 4))

In [193]:
fig, ax = plt.subplots()
ax.plot(volts, chis, label='flopter')
ax.plot(volts, numpy_redchis, label='numpy')
ax.plot(volts, manual_redchis, label='manual')
# ax.plot(volts, chis - scipy_chis, label='scipy')
ax.legend()

In [22]:
fig, ax = plt.subplots()
# ax.plot(volts, chis, label='flopter')
ax.plot(volts, chis - numpy_redchis, label='numpy')
ax.plot(volts, chis - manual_redchis, label='manual')
# ax.plot(volts, chis - scipy_chis, label='scipy')
ax.legend()

---